In [5]:
import pandas as pd
from datetime import date, timedelta

# 어제의 랭크 데이터와 오늘의 랭크 데이터를 읽어서 DF형태로 저장한다.
today_df = pd.read_csv(f"data/{date.today()}-melon.csv", encoding="utf-8")
yesterday_df = pd.read_csv(
    f"data/{date.today()-timedelta(days=1)}-melon.csv", encoding="utf-8"
)

# 두 데이터 프레임을 머지
merged_df = pd.merge(
    today_df,
    yesterday_df,
    on=["title", "artist", "album", "img_url"],
    how="outer",
    suffixes=("_today", "_yesterday"),
)

merged_df

,rank_today,title,artist,album,img_url,rank_yesterday
0,3.0,APT.,로제 (ROSÉ),APT.,https://cdnimg.melon.co.kr/cm2/album/images/11...,3.0
1,47.0,Armageddon,aespa,Armageddon - The 1st Album,https://cdnimg.melon.co.kr/cm2/album/images/11...,58.0
2,72.0,Boom Boom Bass,RIIZE,RIIZING - The 1st Mini Album,https://cdnimg.melon.co.kr/cm2/album/images/11...,79.0
3,68.0,Bubble Gum,NewJeans,How Sweet,https://cdnimg.melon.co.kr/cm2/album/images/11...,91.0
4,77.0,Cherish (My Love),아일릿(ILLIT),I’LL LIKE YOU,https://cdnimg.melon.co.kr/cm2/album/images/11...,92.0
...,...,...,...,...,...,...
99,50.0,클락션 (Klaxon),(여자)아이들,I SWAY,https://cdnimg.melon.co.kr/cm2/album/images/11...,67.0
100,84.0,파이팅 해야지 (Feat. 이영지),부석순 (SEVENTEEN),부석순 1st Single Album 'SECOND WIND',https://cdnimg.melon.co.kr/cm2/album/images/11...,99.0
101,20.0,한 페이지가 될 수 있게,DAY6 (데이식스),The Book of Us : Gravity,https://cdnimg.melon.co.kr/cm/album/images/103...,20.0
102,52.0,해야 (HEYA),IVE (아이브),IVE SWITCH,https://cdnimg.melon.co.kr/cm2/album/images/11...,61.0


In [6]:
# 결측값 채우기, 타입변환
merged_df = merged_df.fillna(101)
merged_df["rank_today"] = merged_df["rank_today"].astype(int)
merged_df["rank_yesterday"] = merged_df["rank_yesterday"].astype(int)

# 어제 랭크와 오늘 랭크의 차를 계산해서 새로운 컬럼 생성(rank_change)
merged_df["rank_change"] = merged_df["rank_yesterday"] - merged_df["rank_today"]
merged_df = (
    merged_df.rename(columns={"rank_today": "rank"})
    .drop(columns="rank_yesterday")
    .set_index("rank")
)
merged_df

,title,artist,album,img_url,rank_change
rank,,,,,
3,APT.,로제 (ROSÉ),APT.,https://cdnimg.melon.co.kr/cm2/album/images/11...,0
47,Armageddon,aespa,Armageddon - The 1st Album,https://cdnimg.melon.co.kr/cm2/album/images/11...,11
72,Boom Boom Bass,RIIZE,RIIZING - The 1st Mini Album,https://cdnimg.melon.co.kr/cm2/album/images/11...,7
68,Bubble Gum,NewJeans,How Sweet,https://cdnimg.melon.co.kr/cm2/album/images/11...,23
77,Cherish (My Love),아일릿(ILLIT),I’LL LIKE YOU,https://cdnimg.melon.co.kr/cm2/album/images/11...,15
...,...,...,...,...,...
50,클락션 (Klaxon),(여자)아이들,I SWAY,https://cdnimg.melon.co.kr/cm2/album/images/11...,17
84,파이팅 해야지 (Feat. 이영지),부석순 (SEVENTEEN),부석순 1st Single Album 'SECOND WIND',https://cdnimg.melon.co.kr/cm2/album/images/11...,15
20,한 페이지가 될 수 있게,DAY6 (데이식스),The Book of Us : Gravity,https://cdnimg.melon.co.kr/cm/album/images/103...,0


In [7]:
# 컬럼의 순서를 바꾸기 위해 다시 대입해주었음 그리고 index(rank)를 기준으로 정렬
merged_df = merged_df[
    ["rank_change", "title", "artist", "album", "img_url"]
].sort_index()

# 100순위 밑으로 밀려난 데이터 제거
merged_df = merged_df[merged_df.index <= 100]
merged_df

,rank_change,title,artist,album,img_url
rank,,,,,
1,0,"HOME SWEET HOME (feat. 태양, 대성)",G-DRAGON,"HOME SWEET HOME (feat. 태양, 대성)",https://cdnimg.melon.co.kr/cm2/album/images/11...
2,0,Whiplash,aespa,Whiplash - The 5th Mini Album,https://cdnimg.melon.co.kr/cm2/album/images/11...
3,0,APT.,로제 (ROSÉ),APT.,https://cdnimg.melon.co.kr/cm2/album/images/11...
4,0,나는 반딧불,황가람,나는 반딧불,https://cdnimg.melon.co.kr/cm2/album/images/11...
5,0,REBEL HEART,IVE (아이브),IVE EMPATHY,https://cdnimg.melon.co.kr/cm2/album/images/11...
...,...,...,...,...,...
96,-47,Do or Die,임영웅,Do or Die,https://cdnimg.melon.co.kr/cm2/album/images/11...
97,4,눈사람,정승환,그리고 봄,https://cdnimg.melon.co.kr/cm/album/images/101...
98,-43,London Boy,임영웅,Polaroid,https://cdnimg.melon.co.kr/cm2/album/images/11...
